In [ ]:
!pip install librosa numpy pandas tensorflow scikit-learn matplotlib

import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers



  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.5.0.post1-cp312-abi3-win_amd64.whl.metadata (5.6 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached standard_aifc-3.13.0-py3-none-any.whl.metadata (969 bytes)
  Using cached standard_sunau-3.13.0-py3-none-any.whl.metadata (914 bytes)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10

In [ ]:
# read the csv file
df=pd.read_csv("data\Bee_hive.csv")
# to remove the spaces 
df.columns = df.columns.str.strip()
print(df.columns)
df.head()




Index(['file_name', 'Sound_Type', 'Queen', 'Productivity', 'Health'], dtype='object')


,file_name,Sound_Type,Queen,Productivity,Health
0,bike_1.wav,Other Sound,No queen,its productivity is not known because its not ...,Other
1,NO_QueenBee_2.wav,Bee Sound,No queen,unknown,unknown
2,queenbee_1.wav,Bee Sound,Queen Present,unknown,unknown


In [ ]:

# to check the total number of audio clip available in audio folder
audio_folder = "data/audio"
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]

print(f"Total audio files found: {len(audio_files)}")
print(audio_files)  # prints all filenames


Total audio files found: 10
['2022-06-06--02-33-38_2__segment2.wav', '2022-06-06--06-30-21_2__segment2.wav', 'Bike_1.wav', 'NO_QueenBee_2.wav', 'NO_QueenBee_3.wav', 'Queenbee_1.wav', 'QueenBee_2.wav', 'QueenBee_3.wav', 'QueenBee_4.wav', 'QueenBee_5.wav']


In [ ]:
#  set audio parameters
SR = 22050      #Sampling rate
DURATION = 10    #duration of each audio clip in seconds
TARGET_LEN= SR*DURATION    #total samples for fixed-length audio

In [14]:
# load and fix audio length
def load_and_fix(audio_path):
    y,_=librosa.load(audio_path,sr=SR)    # load audio
    if len(y)<TARGET_LEN:
        y=np.pad(y,(0,TARGET_LEN-len(y))) # pad with zeros if too short
    else:
        y=y[:TARGET_LEN] #trim if too long
    return y      
audio_path = "data/audio/Queenbee_1.wav"
y_fixed = load_and_fix(audio_path)
print(len(y_fixed))  # This will always print TARGET_LEN


220500


In [15]:
# Extraction Features(MFCCs)
def extract_features(y):
    # y:fixed length audio waveform
    mfccs=librosa.feature.mfcc(y=y,sr=SR,n_mfcc=20)  # 20 MFCC Features
    return np.mean(mfccs.T,axis=0)  #average over time,result shape:(20,)


In [47]:
# apply feature extraction to all clips
X=[]   # features
y_health=[]   #labels for health
y_queen=[]      #labels for queen presence
y_Prod=[]      # labels for productivity
y_sound_type = []  # labels for bee sound or not
for _, row in df.iterrows():
    # file_path = f"data/audio/{row['file_name']}"
    file_path = os.path.join("data/audio", row['file_name'])
    # Load and fix audio length
    y_wave=load_and_fix(file_path)
    #extract features
    feat=extract_features(y_wave)
    X.append(feat)
   #convert labels to 0/1
    y_health.append(0 if row['Health']=="healthy" else 1)
    y_queen.append(0 if row['Queen']=="absent" else 1) 
    y_Prod.append(0 if row['Productivity']=="low" else 1)
    y_sound_type.append(1 if row['Sound_Type']=="bee" else 0)   # 1 = bee, 0 = not bee  
 # convert list to numpy arrays
X= np.array(X)
y_health=np.array(y_health)
y_queen=np.array(y_queen)
y_Prod=np.array(y_Prod)
y_sound_type=np.array(y_sound_type)
print("Feature Shape:",X.shape)
    

Feature Shape: (3, 20)
